In [2]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from PyPDF2 import PdfReader

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
astra_db_app_token= os.environ['ASTRA_DB_APPLICATION_TOKEN']
astra_db_id=os.environ['ASTRA_DB_ID']

pdfreader=PdfReader("documents/starrnnt.pdf")

In [6]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content
raw_text

'Final Project: RNN-Transducer-based\nLosses for Speech Recognition on\nNoisy Targets\nVladimir Bataev\nUniversity of London\n10 March 2024arXiv:2504.06963v1  [eess.AS]  9 Apr 2025Contents\n1 Project Concept and Motivation 3\n1.1 Overview and Template . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3\n1.2 Motivation . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3\n1.3 Related Work: Systems . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3\n1.4 Related Work: Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\n1.5 Related Work: Losses . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\n2 Literature Review 5\n2.1 Introduction . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n2.2 CTC and Modifications . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n2.2.1 Connectionist Temporal Classification (CTC) loss . . . . . . . . . . . 5\n2.2.2 Wild-card CTC (W-CTC) . . . . . . . . . . 

In [7]:
##Initialize the connection to your database:
cassio.init(token=astra_db_app_token,database_id=astra_db_id)

In [10]:
## Create the LangChain embedding and LLM objects for later usage:
llm = OpenAI(openai_api_key=os.environ['openai_key'])
embedding = OpenAIEmbeddings(openai_api_key=os.environ['openai_key'])

/var/folders/r7/xbd75fl13wvf24ktcp53cf1h0000gn/T/ipykernel_88321/483216564.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=os.environ['openai_key'])
/var/folders/r7/xbd75fl13wvf24ktcp53cf1h0000gn/T/ipykernel_88321/483216564.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=os.environ['openai_key'])


In [11]:
## create langchain vector db
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [13]:
## convert the text chnks then embeddings
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [14]:
## load the dataset into vector db
astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.
